In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from glob import glob

from sklearn.model_selection import train_test_split
from sklearn import metrics

import cv2
import gc
import os

import tensorflow as tf
from tensorflow import keras
from keras import layers

import warnings
warnings.filterwarnings('ignore')

2023-11-05 23:32:00.828128: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-05 23:32:01.237644: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-05 23:32:01.237676: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-05 23:32:01.240337: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-05 23:32:01.464951: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-05 23:32:01.467477: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [3]:
model = keras.models.load_model("cancer.keras")

2023-11-05 23:32:05.201463: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-05 23:32:05.201968: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2211] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [5]:
data_path = "lung-cancer-dataset/The-IQ-OTHNCCD-lung-cancer-dataset/The IQ-OTHNCCD lung cancer dataset"

classes = os.listdir(data_path)

# Prep data for processing

IMG_SIZE = 512      # 256 x 256 images
SPLIT = 0.2
EPOCHS = 10
BATCH_SIZE = 32

X = []
Y = []

for i, categ in enumerate(classes):
    images = glob(f"{data_path}/{categ}/*.jpg")

    for image in images:
        img = cv2.imread(image)

        X.append(cv2.resize(img, (IMG_SIZE, IMG_SIZE)))
        Y.append(i)

X = np.asarray(X)
one_hot_encoded_Y = pd.get_dummies(Y).values

X_train, X_val, Y_train, Y_val = train_test_split(X, one_hot_encoded_Y,
                                                test_size = SPLIT,
                                                random_state = 2023)

print(X_train.shape, X_val.shape)

Y_pred = model.predict(X_val)
Y_val = np.argmax(Y_val, axis = 1)
Y_pred = np.argmax(Y_pred, axis = 1)

(877, 512, 512, 3) (220, 512, 512, 3)
7/7 [==============================] - 6s 775ms/step


In [6]:
metrics.confusion_matrix(Y_val, Y_pred)

array([[80,  0,  0],
       [ 1, 29,  0],
       [ 6,  6, 98]])

In [7]:
print(metrics.classification_report(Y_val, Y_pred, target_names = classes))

                 precision    recall  f1-score   support

   Normal cases       0.92      1.00      0.96        80
   Bengin cases       0.83      0.97      0.89        30
Malignant cases       1.00      0.89      0.94       110

       accuracy                           0.94       220
      macro avg       0.92      0.95      0.93       220
   weighted avg       0.95      0.94      0.94       220

